<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/v2/04_classification/10_image_classification_project/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Classification Project

In this project, we will build an image classification model and use the model to identify if the lungs pictured indicate that the patient has pneumonia.  The outcome of the model will be true or false for each image.

The [data is hosted on Kaggle](https://www.kaggle.com/rob717/pneumonia-dataset) and consists of 5,863 x-ray images. Each image is classified as 'pneumonia' or 'normal'.

## Ethical Considerations

We will frame the problem as:

> *A global pandemic has broken out and hospitals are overwhelmed. One of the primary symptoms of infected individuals is pneumonia. If a patient can be quickly confirmed to have pneumonia, then they can be quickly quarantined before infecting other patients.*
>
> *The problem is that medical staff are overwhelmed and under-rested. There are often times when there is no trained technician available to read lung x-rays and make the call of pneumonia/normal.*
>
> *We have been given some recent x-rays of patients, along with the pneumonia/normal classification. We've been asked to build a model that can be used by an untrained technician to get an initial pneumonia/normal indication for a patient. If the model indicates that the patient is positive for pneumonia, then the patient will be quarantined until a trained technician can make a final call.*

Discuss some of the ethical considerations of building and using this model. 

* Consider potential bias in the data that we have been provided. 
* Should this model err toward precision or accuracy?
* What are the implications of massively over-classifying patients as having pneumonia?
* What are the implications of massively under-classifying patients as having pneumonia?
* Are there any concerns with untrained technicians making the initial call?

The questions above are prompts. Feel free to bring in other considerations that you might have.

### **Student Solution**

> *Your answer goes here*

---

### Answer Key

As with other ethics question, there are many right answers. We are looking for solid consideration of trade-offs.

Some example solutions for the prompt questions:

**Consider potential bias in the data that we've been provided.**

> *This is recent data from one hospital. What if other hospitals choose to use the model? Would it generalize to non-local patients? Expecially if the x-ray equipment is different across hospitals.*

**Should this model err toward precision or accuracy?**

> *This is a tough call because one can overwhelm the isolation spaces while the other could speed up the spread of the pandemic. A higher recall would be preferred if the pandemic was highly contagious. A higher precision might be preferred if the hospital was already out of space.*

**What are the implications of massively over-classifying patients as having pneumonia?**

> *Overwhelmed hospitals.*

**What are the implications of massively under-classifying patients as having pneumonia?**

> *Larger than necessary disease spread.*

**Are there any concerns with untrained technicians making the initial call?**

> *The techs might lean too much on the model and not make "expert decisions" on their own. They would likely trust the model too much.*


---

## Modeling

In this section of the lab, you will build, train, test, and validate a model or models. The data is the ["Detecting Pneumonia" dataset](https://www.kaggle.com/rob717/pneumonia-dataset). You will build a binary classifier that determines if an x-ray image has pneumonia or not.

You'll need to:

* Download the dataset
* Perform EDA on the dataset
* Build a model that can classify the data
* Train the model using the training portion of the dataset (it is already split out)
* Test at least three different models or model configurations using the testing portion of the dataset. This can include changing model types, adding and removing layers or nodes from a neural network, or any other parameter tuning that you find potentially useful. Score the model (using accuracy, precision, recall, F1, or some other relevant score(s)) for each configuration.
* After finding the "best" model and parameters, use the validation portion of the dataset to perform one final sanity check by scoring the model once more with the hold-out data.
* If you train a neural network (or other model that you can get epoch-per-epoch performance), graph that performance over each epoch.

Explain your work!

> *Note: You'll likely want to [enable GPU in this lab](https://colab.research.google.com/notebooks/gpu.ipynb), if it is not already enabled.*

If you get to a working solution that you are happy with and want another challenge, you'll find pre-trained models on the [landing page of the dataset](https://www.kaggle.com/paultimothymooney/detecting-pneumonia-in-x-ray-images). Try to load one of those and see how it compares to your best model.

Use as many text and code cells as you need to for your solution.

### **Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

After uploading our `kaggle.json` file into the colab, we move it into place.

In [0]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

Then we download the data.

In [0]:
! kaggle datasets download paultimothymooney/chest-xray-pneumonia/kernels
! ls

And extract it.

In [0]:
import os
import zipfile

zipfile.ZipFile('chest-xray-pneumonia.zip').extractall()
os.listdir('./')

We search all of the directories to see what files we are dealing with. All end up being `jpeg` files. However, they are triplicated in the version of the data that we tested with.

In [0]:
import collections

for directory, _, files in os.walk('chest_xray'):
  extensions = collections.defaultdict(int)
  for file_name in files:
    extensions[file_name[file_name.rindex('.') + 1:]] += 1
  print(directory, extensions)

Set up some useful variables.

In [0]:
training_dir = 'chest_xray/train'
testing_dir = 'chest_xray/test'
validation_dir = 'chest_xray/val'

Build and score our first model.

In [0]:
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', 
                           input_shape=(256, 256 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

train_data_gen = ImageDataGenerator().flow_from_directory(
    directory=training_dir)

history = model.fit(
    train_data_gen,
    epochs=200,
    steps_per_epoch=200,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss')]
)

testing_image_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory=testing_dir,
    image_data_generator=None)

predictions = model.predict(testing_image_iterator)
predicted_classes = [int(prediction > 0.5) for prediction in predictions.flatten()]

actual_classes = testing_image_iterator.classes

f1_score(actual_classes, predicted_classes)

Plot the training accuracy.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['accuracy']))),
         history.history['accuracy'])
plt.show()

And loss.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['loss']))),
         history.history['loss'])
plt.show()

Add a few more layers and score again.

In [0]:
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu',
                           input_shape=(256, 256 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    # New layers below
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    # New layers above

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

train_data_gen = ImageDataGenerator().flow_from_directory(
    directory=training_dir)

history = model.fit(
    train_data_gen,
    epochs=200,
    steps_per_epoch=200,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss')]
)

testing_image_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory=testing_dir,
    image_data_generator=None)

predictions = model.predict(testing_image_iterator)
predicted_classes = [int(prediction > 0.5) for prediction in predictions.flatten()]

actual_classes = testing_image_iterator.classes

f1_score(actual_classes, predicted_classes)

Plot the training accuracy.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['accuracy']))),
         history.history['accuracy'])
plt.show()

And loss.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['loss']))),
         history.history['loss'])
plt.show()

Try using a different threshold for our classifier and score that.

In [0]:
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu',
                           input_shape=(256, 256 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

train_data_gen = ImageDataGenerator().flow_from_directory(
    directory=training_dir)

history = model.fit(
    train_data_gen,
    epochs=200,
    steps_per_epoch=200,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss')]
)

testing_image_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory=testing_dir,
    image_data_generator=None)

predictions = model.predict(testing_image_iterator)

# Higher threshold below
predicted_classes = [int(prediction > 0.6) for prediction in predictions.flatten()]
# Higher threshold above

actual_classes = testing_image_iterator.classes

f1_score(actual_classes, predicted_classes)

Plot the training accuracy.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['accuracy']))),
         history.history['accuracy'])
plt.show()

And loss.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['loss']))),
         history.history['loss'])
plt.show()

Re-score our best model against the validation dataset.

In [0]:
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256, 256 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

train_data_gen = ImageDataGenerator().flow_from_directory(
    directory=training_dir)

history = model.fit(
    train_data_gen,
    epochs=200,
    steps_per_epoch=200,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss')]
)

# Using validation data below instead of test
validation_image_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory=validation_dir,
    image_data_generator=None)

predictions = model.predict(validation_image_iterator)
predicted_classes = [int(prediction > 0.6) for prediction in predictions.flatten()]

actual_classes = validation_image_iterator.classes
# Using validation data above instead of test

f1_score(actual_classes, predicted_classes)

We then plot the training accuracy one last time.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['accuracy']))),
         history.history['accuracy'])
plt.show()

We plot the training loss one last time.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(history.history['loss']))),
         history.history['loss'])
plt.show()

---